In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pyautogui
import time
import re

In [2]:
ops=webdriver.ChromeOptions()
ops.add_argument("--disable-notifications")

In [3]:
srv=Service("C:\py_selenium\chromedriver_win32\chromedriver.exe")
driver=webdriver.Chrome(service=srv,options=ops)

Incompatible release of chromedriver (version 113.0.5672.63) detected in PATH: C:\webdrivers\chromedriver.exe


In [4]:
driver.get("https://www.bbcgoodfood.com/")
driver.maximize_window()

In [5]:
# import time
# time.sleep(5)
# driver.find_element(By.XPATH,"//button[text()='AGREE']").click()

In [9]:
# alert=driver.switch_to.alert
# print(alert.text)
screen_width, screen_height = pyautogui.size()
x = int(screen_width * 0.6)  # Adjust as needed
y = int(screen_height * 0.75)  # Adjust as needed

pyautogui.moveTo(x, y)
pyautogui.click()
time.sleep(2)

In [10]:
from bs4 import BeautifulSoup
import requests

In [11]:
def func_time(ul_element):
    first_li = ul_element.find_all('li')[0]

    prep_time_ul = first_li.find('ul', class_='list')

    prep_ele = prep_time_ul.find('li')
    # prep_text = prep_ele.get_text(separator=' ', strip=True) if prep_ele else None
    prep_text = prep_ele.find('time').get_text(strip=True) if prep_ele else None
    # print(prep_text)

    cook_ele = prep_ele.find_next_sibling('li')
    # cook_text = cook_ele.get_text(separator=' ', strip=True) if cook_ele else None
    cook_time = cook_ele.find('time') if cook_ele else None
    cook_text = cook_time.get_text(strip=True) if cook_time else None
    # print(cook_text)
    return [('Prep', prep_text), ('Cook', cook_text)]



def func_difficulty(ul_element):
    if ul_element.find('div', class_='post-header__skill-level'):
        return ul_element.find('div', class_='post-header__skill-level').get_text(strip=True)
    else:
        return None

    
def func_serves(ul_element):
    if ul_element.find('div', class_='post-header__servings'):
        return ul_element.find('div', class_='post-header__servings').get_text(strip=True)
    else:
        return None

def func_nutritional_vals(doc):
    nutrition_table = doc.find('table', class_="key-value-blocks")
    #print(nutrition_table.caption.string)

    tbodies = nutrition_table.find_all('tbody')
    nutrition_values = []

    for tbody in tbodies:
        for row in tbody.find_all('tr'):
            tdata = row.find_all('td')
            if len(tdata) > 2:
                nutrition_values.append((tdata[1].get_text(strip=True), tdata[-1].get_text(strip=True)))
            elif len(tdata) == 2:
                nutrition_values.append((tdata[0].get_text(strip=True), tdata[-1].get_text(strip=True)))
    #print(nutrition_values)
    return nutrition_values
                

def func_rating(doc):
    if not doc.find('div', class_='rating__values'):
        return None

    rating_string = doc.find('div', class_='rating__values').select('span.sr-only')[0].get_text(strip=True)
    rating_match = re.findall(r'\d+(?:\.\d+)?', rating_string)
    rating = float(rating_match[0]) if rating_match else None
    # print(rating)

    rating_count = doc.select('span.rating__count-text')[0].get_text(strip=True)
    # print(rating_count)
    return rating,rating_count

In [12]:
def func_scrape_dets(url):
    response = requests.get(url)
    doc = BeautifulSoup(response.content, "html.parser")
    title = doc.find(class_="heading-1")
    print(title.text)
    
    #func_scrape_dets(parent[0])
    
    ul_element = doc.find('ul', class_="post-header__planning")
    #ul_children = ul_element.children
    
    #li_elements = [child for child in ul_children if child.name == 'li']
    
    #getting prep and cook time
    time = func_time(ul_element)
    
    #getting difficulty
    difficulty = func_difficulty(ul_element)
    
    #getting serves
    serves = func_serves(ul_element)
    
    #getting nutritional values
    nutritions = func_nutritional_vals(doc)
    
    #getting ratings for popularity        
    rating = func_rating(doc)
    
    print(time)
    print(difficulty)
    print(serves)
    print(nutritions)
    print(rating)

In [13]:
# func_scrape_dets(parent[0])

In [14]:
func_scrape_dets("https://www.bbcgoodfood.com/recipes/chicken-chorizo-jambalaya")

Chicken & chorizo jambalaya
[('Prep', '10 mins'), ('Cook', '45 mins')]
Easy
Serves 4
[('kcal', '445'), ('fat', '10g'), ('saturates', '3g'), ('carbs', '64g'), ('sugars', '7g'), ('fibre', '2g'), ('protein', '30g'), ('salt', '1.2g')]
(4.8, '2545 ratings')


In [15]:
ingredients=["chicken", "mushroom", "fish", "egg", "paneer", "beef", "soy bean"]

In [16]:
parent=[]

def func_getItems(driver):
    item=driver.find_elements(By.CLASS_NAME,"heading-4")
    item=item[1:len(item)-1]
    for i in range(len(item)):
        parent.append(item[i].find_element(By.XPATH,"./parent::a").get_attribute('href'))

In [17]:
for ingredient in ingredients:
    searchbox=driver.find_element(By.CLASS_NAME,"form-input")
    searchbox.clear()
    searchbox.send_keys(ingredient)
    searchbox.submit()
    func_getItems(driver)
    driver.back()

In [18]:
len(parent)

210

In [19]:
#their parent a tag which has the href
# parent=[]
# for i in range(len(item)):
#     parent.append(item[i].find_element(By.XPATH,"./parent::a").get_attribute('href'))

In [20]:
parent

['https://www.bbcgoodfood.com/recipes/chicken-chorizo-jambalaya',
 'https://www.bbcgoodfood.com/recipes/chicken-noodle-soup',
 'https://www.bbcgoodfood.com/recipes/chicken-pasta-bake',
 'https://www.bbcgoodfood.com/recipes/chicken-jalfrezi',
 'https://www.bbcgoodfood.com/recipes/chinese-chicken-curry',
 'https://www.bbcgoodfood.com/recipes/chicken-satay-salad',
 'https://www.bbcgoodfood.com/recipes/chicken-arrabbiata-stew-parmesan-dumplings',
 'https://www.bbcgoodfood.com/recipes/chicken-white-bean-stew',
 'https://www.bbcgoodfood.com/recipes/chicken-chorizo-ragu',
 'https://www.bbcgoodfood.com/recipes/summer-winter-chicken',
 'https://www.bbcgoodfood.com/recipes/mustard-stuffed-chicken',
 'https://www.bbcgoodfood.com/recipes/chicken-biryani',
 'https://www.bbcgoodfood.com/recipes/easy-chicken-curry',
 'https://www.bbcgoodfood.com/recipes/healthier-chicken-balti',
 'https://www.bbcgoodfood.com/recipes/chicken-and-bean-enchiladas',
 'https://www.bbcgoodfood.com/recipes/home-style-chicke

In [21]:
for i in parent:
    func_scrape_dets(i)

Chicken & chorizo jambalaya
[('Prep', '10 mins'), ('Cook', '45 mins')]
Easy
Serves 4
[('kcal', '445'), ('fat', '10g'), ('saturates', '3g'), ('carbs', '64g'), ('sugars', '7g'), ('fibre', '2g'), ('protein', '30g'), ('salt', '1.2g')]
(4.8, '2545 ratings')
Chicken noodle soup
[('Prep', '10 mins'), ('Cook', '30 mins')]
Easy
Serves 2
[('kcal', '217'), ('fat', '2g'), ('saturates', '0.4g'), ('carbs', '26g'), ('sugars', '1g'), ('fibre', '0.6g'), ('protein', '26g'), ('salt', '2.5g')]
(4.8, '751 ratings')
Chicken pasta bake
[('Prep', '30 mins'), ('Cook', '45 mins')]
Easy
Serves 6
[('kcal', '575'), ('fat', '30g'), ('saturates', '14g'), ('carbs', '41g'), ('sugars', '9g'), ('fibre', '5g'), ('protein', '33g'), ('salt', '0.5g')]
(4.7, '592 ratings')
Chicken jalfrezi
[('Prep', '10 mins'), ('Cook', '1 hr')]
Easy
Serves 4
[('kcal', '252'), ('fat', '8g'), ('saturates', '1g'), ('carbs', '11g'), ('sugars', '9g'), ('fibre', '5g'), ('protein', '30g'), ('salt', '0.4g')]
(4.5, '459 ratings')
Chinese chicken cur

Creamy mushroom pasta
[('Prep', '15 mins'), ('Cook', '25 mins')]
Easy
Serves 4
[('kcal', '801'), ('fat', '52g'), ('saturates', '30g'), ('carbs', '45g'), ('sugars', '4g'), ('fibre', '5g'), ('protein', '29g'), ('salt', '0.97g')]
(4.3, '128 ratings')
Chicken and mushrooms
[('Prep', '15 mins'), ('Cook', '25 mins')]
Easy
Serves 4
[('kcal', '260'), ('fat', '13g'), ('saturates', '3g'), ('carbs', '3g'), ('sugars', '1g'), ('fibre', '3g'), ('protein', '32g'), ('salt', '0.9g')]
(4.5, '294 ratings')
Chicken & mushroom puff pie
[('Prep', '45 mins'), ('Cook', '30 mins')]
Easy
Serves 4 - 6
[('kcal', '855'), ('fat', '47g'), ('saturates', '17g'), ('carbs', '57g'), ('sugars', '6g'), ('fibre', '1g'), ('protein', '55g'), ('salt', '2.7g')]
(4.8, '431 ratings')
Mushroom & spinach risotto
[('Prep', '50 mins'), ('Cook', None)]
More effort
Serves 2
[('kcal', '574'), ('fat', '22g'), ('saturates', '10g'), ('carbs', '70g'), ('sugars', '0g'), ('fibre', '5g'), ('protein', '17g'), ('salt', '1.93g')]
(4.8, '279 ratin

Greek-style roast fish
[('Prep', '10 mins'), ('Cook', '50 mins')]
Easy
Serves 2
[('kcal', '388'), ('fat', '13g'), ('saturates', '2g'), ('carbs', '42g'), ('sugars', '11g'), ('fibre', '6g'), ('protein', '23g'), ('salt', '0.4g')]
(4.3, '151 ratings')
Creamy fish & leek pie
[('Prep', '20 mins'), ('Cook', '1 hr and 45 mins')]
Easy
Serves 4-6
[('kcal', '833'), ('fat', '60g'), ('saturates', '35g'), ('carbs', '42g'), ('sugars', '6g'), ('fibre', '6g'), ('protein', '24g'), ('salt', '1.2g')]
(4.7, '61 ratings')
Ultimate fish cakes
[('Prep', '25 mins'), ('Cook', '30 mins')]
More effort
Serves 4
[('kcal', '359'), ('fat', '12g'), ('saturates', '2g'), ('carbs', '36g'), ('sugars', '3g'), ('fibre', '2g'), ('protein', '29g'), ('salt', '0.7g')]
(4.7, '165 ratings')
Thai-style steamed fish
[('Prep', '10 mins'), ('Cook', '15 mins')]
Easy
Serves 2
[('kcal', '199'), ('fat', '7g'), ('saturates', '2g'), ('carbs', '4g'), ('sugars', '0g'), ('fibre', '0g'), ('protein', '29g'), ('salt', '3.25g')]
(4.5, '222 rating

Curried spinach, eggs & chickpeas
[('Prep', '15 mins'), ('Cook', '35 mins')]
Easy
Serves 2
[('kcal', '469'), ('fat', '20g'), ('saturates', '4g'), ('carbs', '39g'), ('sugars', '18g'), ('fibre', '12g'), ('protein', '28g'), ('salt', '0.5g')]
(4.9, '27 ratings')
Egg custard
[('Prep', '10 mins'), ('Cook', '40 mins')]
Easy
Serves 4
[('kcal', '434'), ('fat', '36g'), ('saturates', '21g'), ('carbs', '18g'), ('sugars', '18g'), ('fibre', '0g'), ('protein', '9g'), ('salt', '0.3g')]
(4.8, '16 ratings')
Easy egg-fried rice
[('Prep', '10 mins'), ('Cook', '10 mins')]
Easy
Serves 4
[('kcal', '387'), ('fat', '14g'), ('saturates', '2g'), ('carbs', '53g'), ('sugars', '2g'), ('fibre', '1g'), ('protein', '12g'), ('salt', '0.2g')]
(3.8, '86 ratings')
Sweet potato cakes with poached eggs
[('Prep', '15 mins'), ('Cook', '15 mins')]
More effort
Serves 4
[('kcal', '507'), ('fat', '15g'), ('saturates', '4g'), ('carbs', '72g'), ('sugars', '23g'), ('fibre', '7g'), ('protein', '18g'), ('salt', '0.9g')]
(4.3, '67 rati

Paneer chutney pakora
[('Prep', '25 mins'), ('Cook', '15 mins')]
Easy
Serves 4 as a starter
[('kcal', '531'), ('fat', '36g'), ('saturates', '17g'), ('carbs', '17g'), ('sugars', '2g'), ('fibre', '3g'), ('protein', '33g'), ('salt', '2g')]
(5.0, '1 rating')
Spiced paneer
[('Prep', '50 mins'), ('Cook', None)]
Easy
Serves 4 as a main, or 6-8 as a side dish
[('kcal', '463'), ('fat', '34g'), ('saturates', '16g'), ('carbs', '16g'), ('sugars', '3g'), ('fibre', '2g'), ('protein', '24g'), ('salt', '3.75g')]
(4.0, '30 ratings')
Paneer & chickpea pulao
[('Prep', '15 mins'), ('Cook', '40 mins')]
Easy
Serves 6
[('kcal', '414'), ('fat', '16g'), ('saturates', '7g'), ('carbs', '48g'), ('sugars', '4g'), ('fibre', '5g'), ('protein', '18g'), ('salt', '1.3g')]
(4.0, '5 ratings')
Sticky paneer mango naans with herby yogurt
[('Prep', '20 mins'), ('Cook', '5 mins')]
Easy
Serves 1
[('kcal', '1023'), ('fat', '44g'), ('saturates', '21g'), ('carbs', '101g'), ('sugars', '28g'), ('fibre', '11g'), ('protein', '50g'),

Beef tacos
[('Prep', '10 mins'), ('Cook', '30 mins')]
More effort
Serves 3
[('kcal', '792'), ('fat', '44g'), ('saturates', '16g'), ('carbs', '51g'), ('sugars', '8g'), ('fibre', '9g'), ('protein', '44g'), ('salt', '1.7g')]
(4.6, '29 ratings')
Next level beef stroganoff
[('Prep', '10 mins'), ('Cook', '25 mins')]
Easy
Serves 4
[('kcal', '438'), ('fat', '34g'), ('saturates', '20g'), ('carbs', '3g'), ('sugars', '3g'), ('fibre', '2g'), ('protein', '30g'), ('salt', '2.1g')]
(4.4, '55 ratings')
Braised beef with ginger
[('Prep', '35 mins'), ('Cook', '3 hrs')]
Easy
Serves 6
[('kcal', '405'), ('fat', '11g'), ('saturates', '4g'), ('carbs', '26g'), ('sugars', '23g'), ('fibre', '1g'), ('protein', '51g'), ('salt', '3.96g')]
(4.8, '124 ratings')
Chinese-style braised beef one-pot
[('Prep', '10 mins'), ('Cook', '2 hrs')]
Easy
Serves 6
[('kcal', '513'), ('fat', '29g'), ('saturates', '10g'), ('carbs', '9g'), ('sugars', '4g'), ('fibre', '0g'), ('protein', '54g'), ('salt', '2.39g')]
(4.7, '254 ratings')
H

Spicy chilli bean soup
[('Prep', '10 mins'), ('Cook', '40 mins')]
Easy
Serves 4
[('kcal', '157'), ('fat', '4g'), ('saturates', '0g'), ('carbs', '18g'), ('sugars', '10g'), ('fibre', '9g'), ('protein', '8g'), ('salt', '0.5g')]
(4.8, '49 ratings')
Slow-cooked soy-glazed chicken
[('Prep', '30 mins'), ('Cook', '2 hrs')]
Easy
Serves 4 - 6
[('kcal', '439'), ('fat', '24g'), ('saturates', '6g'), ('carbs', '6g'), ('sugars', '5g'), ('fibre', '2g'), ('protein', '47g'), ('salt', '2.4g')]
(4.9, '17 ratings')
Soy & chilli chicken with peppers & peanuts
[('Prep', '15 mins'), ('Cook', '15 mins')]
Easy
Serves 4
[('kcal', '412'), ('fat', '23g'), ('saturates', '3g'), ('carbs', '12g'), ('sugars', '7g'), ('fibre', '4g'), ('protein', '36g'), ('salt', '1.6g')]
(4.9, '13 ratings')
Mexican bean soup with guacamole
[('Prep', '10 mins'), ('Cook', '20 mins')]
Easy
Serves 2
[('kcal', '391'), ('fat', '15g'), ('saturates', '3g'), ('carbs', '38g'), ('sugars', '18g'), ('fibre', '20g'), ('protein', '15g'), ('salt', '0.4

In [22]:
df1=[]

for url in parent:
    response = requests.get(url)
    doc = BeautifulSoup(response.content, "html.parser")
    title = doc.find(class_="heading-1")
    print(title.text)
    
    #func_scrape_dets(parent[0])
    
    ul_element = doc.find('ul', class_="post-header__planning")
    #ul_children = ul_element.children
    
    #li_elements = [child for child in ul_children if child.name == 'li']
    
    #getting prep and cook time
    time = func_time(ul_element)
    
    #getting difficulty
    difficulty = func_difficulty(ul_element)
    
    #getting serves
    serves = func_serves(ul_element)
    
    #getting nutritional values
    nutritions = func_nutritional_vals(doc)
    
    #getting ratings for popularity        
    rating = func_rating(doc)
    
    df1.append({'Item': title.text, 'Time': time, 'Difficulty': difficulty, 'Serves': serves, 'Nutritional_Values': nutritions, 'Rating': rating})

Chicken & chorizo jambalaya
Chicken noodle soup
Chicken pasta bake
Chicken jalfrezi
Chinese chicken curry
Chicken satay salad
Chicken arrabbiata stew & parmesan dumplings
Easy chicken casserole
Chicken & chorizo ragu
Summer-in-winter chicken
Mustard-stuffed chicken
Chicken biryani
Easy chicken curry
Chicken balti
Chicken enchiladas
Home-style chicken curry
Easy teriyaki chicken
Chicken & chorizo paella
Creamy chicken stew
Moroccan chicken
Easy chicken fajitas
Amatriciana chicken traybake
Chicken madras
Coconut chicken curry
Classic roast chicken & gravy
Chicken souvlaki
Chicken parmigiana
Slow cooker chicken tikka masala
Chicken & bacon pasta
One-pot chicken chasseur
Mushroom risotto
Mushroom stroganoff
Mushroom soup
Creamy mushroom pasta
Chicken and mushrooms
Chicken & mushroom puff pie
Mushroom & spinach risotto
One pot chicken and mushroom risotto
Mushroom & potato curry
Bacon & mushroom pasta
Pan-fried chicken in mushroom sauce
Chicken, kale & mushroom pot pie
Sausage & mushroom ra

In [23]:
import pandas as pd
df1=pd.DataFrame(df1)

In [24]:
df1

,Item,Time,Difficulty,Serves,Nutritional_Values,Rating
0,Chicken & chorizo jambalaya,"[(Prep, 10 mins), (Cook, 45 mins)]",Easy,Serves 4,"[(kcal, 445), (fat, 10g), (saturates, 3g), (ca...","(4.8, 2545 ratings)"
1,Chicken noodle soup,"[(Prep, 10 mins), (Cook, 30 mins)]",Easy,Serves 2,"[(kcal, 217), (fat, 2g), (saturates, 0.4g), (c...","(4.8, 751 ratings)"
2,Chicken pasta bake,"[(Prep, 30 mins), (Cook, 45 mins)]",Easy,Serves 6,"[(kcal, 575), (fat, 30g), (saturates, 14g), (c...","(4.7, 592 ratings)"
3,Chicken jalfrezi,"[(Prep, 10 mins), (Cook, 1 hr)]",Easy,Serves 4,"[(kcal, 252), (fat, 8g), (saturates, 1g), (car...","(4.5, 459 ratings)"
4,Chinese chicken curry,"[(Prep, 15 mins), (Cook, 40 mins)]",Easy,Serves 4,"[(kcal, 264), (fat, 8g), (saturates, 1g), (car...","(4.7, 476 ratings)"
...,...,...,...,...,...,...
205,Butter bean & chorizo stew,"[(Prep, 5 mins), (Cook, 15 mins)]",Easy,Serves 4,"[(kcal, 491), (fat, 32g), (saturates, 8g), (ca...","(4.5, 149 ratings)"
206,Sausage & white bean casserole,"[(Prep, 20 mins), (Cook, 1 hr and 5 mins)]",Easy,Serves 4,"[(kcal, 363), (fat, 17g), (saturates, 6g), (ca...","(4.5, 100 ratings)"
207,Chipotle sweet potato & black bean stew with c...,"[(Prep, 10 mins), (Cook, 50 mins)]",Easy,Serves 4,"[(kcal, 547), (fat, 23g), (saturates, 12g), (c...","(4.8, 132 ratings)"
208,"Salmon with sesame, soy & ginger noodles","[(Prep, 10 mins), (Cook, 20 mins)]",Easy,Serves 2,"[(kcal, 749), (fat, 27g), (saturates, 5g), (ca...","(4.6, 40 ratings)"


In [25]:
df2=df1.copy()

In [26]:
df2

,Item,Time,Difficulty,Serves,Nutritional_Values,Rating
0,Chicken & chorizo jambalaya,"[(Prep, 10 mins), (Cook, 45 mins)]",Easy,Serves 4,"[(kcal, 445), (fat, 10g), (saturates, 3g), (ca...","(4.8, 2545 ratings)"
1,Chicken noodle soup,"[(Prep, 10 mins), (Cook, 30 mins)]",Easy,Serves 2,"[(kcal, 217), (fat, 2g), (saturates, 0.4g), (c...","(4.8, 751 ratings)"
2,Chicken pasta bake,"[(Prep, 30 mins), (Cook, 45 mins)]",Easy,Serves 6,"[(kcal, 575), (fat, 30g), (saturates, 14g), (c...","(4.7, 592 ratings)"
3,Chicken jalfrezi,"[(Prep, 10 mins), (Cook, 1 hr)]",Easy,Serves 4,"[(kcal, 252), (fat, 8g), (saturates, 1g), (car...","(4.5, 459 ratings)"
4,Chinese chicken curry,"[(Prep, 15 mins), (Cook, 40 mins)]",Easy,Serves 4,"[(kcal, 264), (fat, 8g), (saturates, 1g), (car...","(4.7, 476 ratings)"
...,...,...,...,...,...,...
205,Butter bean & chorizo stew,"[(Prep, 5 mins), (Cook, 15 mins)]",Easy,Serves 4,"[(kcal, 491), (fat, 32g), (saturates, 8g), (ca...","(4.5, 149 ratings)"
206,Sausage & white bean casserole,"[(Prep, 20 mins), (Cook, 1 hr and 5 mins)]",Easy,Serves 4,"[(kcal, 363), (fat, 17g), (saturates, 6g), (ca...","(4.5, 100 ratings)"
207,Chipotle sweet potato & black bean stew with c...,"[(Prep, 10 mins), (Cook, 50 mins)]",Easy,Serves 4,"[(kcal, 547), (fat, 23g), (saturates, 12g), (c...","(4.8, 132 ratings)"
208,"Salmon with sesame, soy & ginger noodles","[(Prep, 10 mins), (Cook, 20 mins)]",Easy,Serves 2,"[(kcal, 749), (fat, 27g), (saturates, 5g), (ca...","(4.6, 40 ratings)"


In [27]:
# df2['kcal']=df2['Nutritional_Values'].apply(lambda x:x[1] if x[0]=='kcal')
# df2['fat']=df2['Nutritional_Values'].apply(lambda x:x[1] if x[0]=='fat')
# df2['saturates']=df2['Nutritional_Values'].apply(lambda x:x[1] if x[0]=='saturates')
# df2['carbs']=df2['Nutritional_Values'].apply(lambda x:x[1] if x[0]=='carbs')
# df2['sugars']=df2['Nutritional_Values'].apply(lambda x:x[1] if x[0]=='sugars')
# df2['fibre']=df2['Nutritional_Values'].apply(lambda x:x[1] if x[0]=='fibre')
# df2['protein']=df2['Nutritional_Values'].apply(lambda x:x[1] if x[0]=='protein')
# df2['salt']=df2['Nutritional_Values'].apply(lambda x:x[1] if x[0]=='salt')

In [28]:
df2['Nutritional_Values']

0      [(kcal, 445), (fat, 10g), (saturates, 3g), (ca...
1      [(kcal, 217), (fat, 2g), (saturates, 0.4g), (c...
2      [(kcal, 575), (fat, 30g), (saturates, 14g), (c...
3      [(kcal, 252), (fat, 8g), (saturates, 1g), (car...
4      [(kcal, 264), (fat, 8g), (saturates, 1g), (car...
                             ...                        
205    [(kcal, 491), (fat, 32g), (saturates, 8g), (ca...
206    [(kcal, 363), (fat, 17g), (saturates, 6g), (ca...
207    [(kcal, 547), (fat, 23g), (saturates, 12g), (c...
208    [(kcal, 749), (fat, 27g), (saturates, 5g), (ca...
209    [(kcal, 435), (fat, 15g), (saturates, 5g), (ca...
Name: Nutritional_Values, Length: 210, dtype: object

In [29]:
df2[['kcal', 'fat', 'saturates', 'carbs','sugars','fibre','protein','salt']] = df2['Nutritional_Values'].apply(lambda x: pd.Series(dict(x)))

In [30]:
df2

,Item,Time,Difficulty,Serves,Nutritional_Values,Rating,kcal,fat,saturates,carbs,sugars,fibre,protein,salt
0,Chicken & chorizo jambalaya,"[(Prep, 10 mins), (Cook, 45 mins)]",Easy,Serves 4,"[(kcal, 445), (fat, 10g), (saturates, 3g), (ca...","(4.8, 2545 ratings)",445,10g,3g,64g,7g,2g,30g,1.2g
1,Chicken noodle soup,"[(Prep, 10 mins), (Cook, 30 mins)]",Easy,Serves 2,"[(kcal, 217), (fat, 2g), (saturates, 0.4g), (c...","(4.8, 751 ratings)",217,2g,0.4g,26g,1g,0.6g,26g,2.5g
2,Chicken pasta bake,"[(Prep, 30 mins), (Cook, 45 mins)]",Easy,Serves 6,"[(kcal, 575), (fat, 30g), (saturates, 14g), (c...","(4.7, 592 ratings)",575,30g,14g,41g,9g,5g,33g,0.5g
3,Chicken jalfrezi,"[(Prep, 10 mins), (Cook, 1 hr)]",Easy,Serves 4,"[(kcal, 252), (fat, 8g), (saturates, 1g), (car...","(4.5, 459 ratings)",252,8g,1g,11g,9g,5g,30g,0.4g
4,Chinese chicken curry,"[(Prep, 15 mins), (Cook, 40 mins)]",Easy,Serves 4,"[(kcal, 264), (fat, 8g), (saturates, 1g), (car...","(4.7, 476 ratings)",264,8g,1g,7g,3g,2g,40g,0.7g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Butter bean & chorizo stew,"[(Prep, 5 mins), (Cook, 15 mins)]",Easy,Serves 4,"[(kcal, 491), (fat, 32g), (saturates, 8g), (ca...","(4.5, 149 ratings)",491,32g,8g,24g,10g,8g,23g,2.5g
206,Sausage & white bean casserole,"[(Prep, 20 mins), (Cook, 1 hr and 5 mins)]",Easy,Serves 4,"[(kcal, 363), (fat, 17g), (saturates, 6g), (ca...","(4.5, 100 ratings)",363,17g,6g,32g,15g,11g,16g,1.2g
207,Chipotle sweet potato & black bean stew with c...,"[(Prep, 10 mins), (Cook, 50 mins)]",Easy,Serves 4,"[(kcal, 547), (fat, 23g), (saturates, 12g), (c...","(4.8, 132 ratings)",547,23g,12g,60g,12g,15g,18g,1.5g
208,"Salmon with sesame, soy & ginger noodles","[(Prep, 10 mins), (Cook, 20 mins)]",Easy,Serves 2,"[(kcal, 749), (fat, 27g), (saturates, 5g), (ca...","(4.6, 40 ratings)",749,27g,5g,78g,27g,4g,42g,4g


In [31]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Item                210 non-null    object
 1   Time                210 non-null    object
 2   Difficulty          209 non-null    object
 3   Serves              209 non-null    object
 4   Nutritional_Values  210 non-null    object
 5   Rating              209 non-null    object
 6   kcal                210 non-null    object
 7   fat                 210 non-null    object
 8   saturates           210 non-null    object
 9   carbs               210 non-null    object
 10  sugars              210 non-null    object
 11  fibre               210 non-null    object
 12  protein             210 non-null    object
 13  salt                210 non-null    object
dtypes: object(14)
memory usage: 23.1+ KB


In [32]:
#turning to numeric columns

def change_col_names():
    cols=['fat','saturates','carbs','sugars','fibre','protein','salt']
    for col in cols:
        df2[col]=df2[col].str.replace('g','')

def change_col_type():
    cols=['kcal','fat','saturates','carbs','sugars','fibre','protein','salt']
    for col in cols:
        #df2[col]=df2[col].str.extract('(\d+)', expand = False)
        df2[col]=pd.to_numeric(df2[col])

change_col_names()
change_col_type()

In [33]:
df2

,Item,Time,Difficulty,Serves,Nutritional_Values,Rating,kcal,fat,saturates,carbs,sugars,fibre,protein,salt
0,Chicken & chorizo jambalaya,"[(Prep, 10 mins), (Cook, 45 mins)]",Easy,Serves 4,"[(kcal, 445), (fat, 10g), (saturates, 3g), (ca...","(4.8, 2545 ratings)",445,10.0,3.0,64.0,7.0,2.0,30.0,1.2
1,Chicken noodle soup,"[(Prep, 10 mins), (Cook, 30 mins)]",Easy,Serves 2,"[(kcal, 217), (fat, 2g), (saturates, 0.4g), (c...","(4.8, 751 ratings)",217,2.0,0.4,26.0,1.0,0.6,26.0,2.5
2,Chicken pasta bake,"[(Prep, 30 mins), (Cook, 45 mins)]",Easy,Serves 6,"[(kcal, 575), (fat, 30g), (saturates, 14g), (c...","(4.7, 592 ratings)",575,30.0,14.0,41.0,9.0,5.0,33.0,0.5
3,Chicken jalfrezi,"[(Prep, 10 mins), (Cook, 1 hr)]",Easy,Serves 4,"[(kcal, 252), (fat, 8g), (saturates, 1g), (car...","(4.5, 459 ratings)",252,8.0,1.0,11.0,9.0,5.0,30.0,0.4
4,Chinese chicken curry,"[(Prep, 15 mins), (Cook, 40 mins)]",Easy,Serves 4,"[(kcal, 264), (fat, 8g), (saturates, 1g), (car...","(4.7, 476 ratings)",264,8.0,1.0,7.0,3.0,2.0,40.0,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Butter bean & chorizo stew,"[(Prep, 5 mins), (Cook, 15 mins)]",Easy,Serves 4,"[(kcal, 491), (fat, 32g), (saturates, 8g), (ca...","(4.5, 149 ratings)",491,32.0,8.0,24.0,10.0,8.0,23.0,2.5
206,Sausage & white bean casserole,"[(Prep, 20 mins), (Cook, 1 hr and 5 mins)]",Easy,Serves 4,"[(kcal, 363), (fat, 17g), (saturates, 6g), (ca...","(4.5, 100 ratings)",363,17.0,6.0,32.0,15.0,11.0,16.0,1.2
207,Chipotle sweet potato & black bean stew with c...,"[(Prep, 10 mins), (Cook, 50 mins)]",Easy,Serves 4,"[(kcal, 547), (fat, 23g), (saturates, 12g), (c...","(4.8, 132 ratings)",547,23.0,12.0,60.0,12.0,15.0,18.0,1.5
208,"Salmon with sesame, soy & ginger noodles","[(Prep, 10 mins), (Cook, 20 mins)]",Easy,Serves 2,"[(kcal, 749), (fat, 27g), (saturates, 5g), (ca...","(4.6, 40 ratings)",749,27.0,5.0,78.0,27.0,4.0,42.0,4.0


In [34]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Item                210 non-null    object 
 1   Time                210 non-null    object 
 2   Difficulty          209 non-null    object 
 3   Serves              209 non-null    object 
 4   Nutritional_Values  210 non-null    object 
 5   Rating              209 non-null    object 
 6   kcal                210 non-null    int64  
 7   fat                 210 non-null    float64
 8   saturates           210 non-null    float64
 9   carbs               210 non-null    float64
 10  sugars              210 non-null    float64
 11  fibre               210 non-null    float64
 12  protein             210 non-null    float64
 13  salt                210 non-null    float64
dtypes: float64(7), int64(1), object(6)
memory usage: 23.1+ KB


In [35]:
df2.drop('Nutritional_Values', axis=1, inplace=True)

In [36]:
df2

,Item,Time,Difficulty,Serves,Rating,kcal,fat,saturates,carbs,sugars,fibre,protein,salt
0,Chicken & chorizo jambalaya,"[(Prep, 10 mins), (Cook, 45 mins)]",Easy,Serves 4,"(4.8, 2545 ratings)",445,10.0,3.0,64.0,7.0,2.0,30.0,1.2
1,Chicken noodle soup,"[(Prep, 10 mins), (Cook, 30 mins)]",Easy,Serves 2,"(4.8, 751 ratings)",217,2.0,0.4,26.0,1.0,0.6,26.0,2.5
2,Chicken pasta bake,"[(Prep, 30 mins), (Cook, 45 mins)]",Easy,Serves 6,"(4.7, 592 ratings)",575,30.0,14.0,41.0,9.0,5.0,33.0,0.5
3,Chicken jalfrezi,"[(Prep, 10 mins), (Cook, 1 hr)]",Easy,Serves 4,"(4.5, 459 ratings)",252,8.0,1.0,11.0,9.0,5.0,30.0,0.4
4,Chinese chicken curry,"[(Prep, 15 mins), (Cook, 40 mins)]",Easy,Serves 4,"(4.7, 476 ratings)",264,8.0,1.0,7.0,3.0,2.0,40.0,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Butter bean & chorizo stew,"[(Prep, 5 mins), (Cook, 15 mins)]",Easy,Serves 4,"(4.5, 149 ratings)",491,32.0,8.0,24.0,10.0,8.0,23.0,2.5
206,Sausage & white bean casserole,"[(Prep, 20 mins), (Cook, 1 hr and 5 mins)]",Easy,Serves 4,"(4.5, 100 ratings)",363,17.0,6.0,32.0,15.0,11.0,16.0,1.2
207,Chipotle sweet potato & black bean stew with c...,"[(Prep, 10 mins), (Cook, 50 mins)]",Easy,Serves 4,"(4.8, 132 ratings)",547,23.0,12.0,60.0,12.0,15.0,18.0,1.5
208,"Salmon with sesame, soy & ginger noodles","[(Prep, 10 mins), (Cook, 20 mins)]",Easy,Serves 2,"(4.6, 40 ratings)",749,27.0,5.0,78.0,27.0,4.0,42.0,4.0


In [37]:
df2['Serves']=df2['Serves'].str.extract('(\d+)', expand = False)
df2['Serves']=pd.to_numeric(df2['Serves'], downcast='integer', errors='coerce').fillna(0).astype(int)

In [38]:
df2

,Item,Time,Difficulty,Serves,Rating,kcal,fat,saturates,carbs,sugars,fibre,protein,salt
0,Chicken & chorizo jambalaya,"[(Prep, 10 mins), (Cook, 45 mins)]",Easy,4,"(4.8, 2545 ratings)",445,10.0,3.0,64.0,7.0,2.0,30.0,1.2
1,Chicken noodle soup,"[(Prep, 10 mins), (Cook, 30 mins)]",Easy,2,"(4.8, 751 ratings)",217,2.0,0.4,26.0,1.0,0.6,26.0,2.5
2,Chicken pasta bake,"[(Prep, 30 mins), (Cook, 45 mins)]",Easy,6,"(4.7, 592 ratings)",575,30.0,14.0,41.0,9.0,5.0,33.0,0.5
3,Chicken jalfrezi,"[(Prep, 10 mins), (Cook, 1 hr)]",Easy,4,"(4.5, 459 ratings)",252,8.0,1.0,11.0,9.0,5.0,30.0,0.4
4,Chinese chicken curry,"[(Prep, 15 mins), (Cook, 40 mins)]",Easy,4,"(4.7, 476 ratings)",264,8.0,1.0,7.0,3.0,2.0,40.0,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Butter bean & chorizo stew,"[(Prep, 5 mins), (Cook, 15 mins)]",Easy,4,"(4.5, 149 ratings)",491,32.0,8.0,24.0,10.0,8.0,23.0,2.5
206,Sausage & white bean casserole,"[(Prep, 20 mins), (Cook, 1 hr and 5 mins)]",Easy,4,"(4.5, 100 ratings)",363,17.0,6.0,32.0,15.0,11.0,16.0,1.2
207,Chipotle sweet potato & black bean stew with c...,"[(Prep, 10 mins), (Cook, 50 mins)]",Easy,4,"(4.8, 132 ratings)",547,23.0,12.0,60.0,12.0,15.0,18.0,1.5
208,"Salmon with sesame, soy & ginger noodles","[(Prep, 10 mins), (Cook, 20 mins)]",Easy,2,"(4.6, 40 ratings)",749,27.0,5.0,78.0,27.0,4.0,42.0,4.0
